In [39]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults
import numpy as np

show_object = show

In [40]:
set_sidecar("Gears")

In [41]:
enable_replay()
show_object = replay


Enabling jupyter_cadquery replay


In [42]:
set_defaults(axes=True, grid=True, axes0=True, edge_accuracy=0.001, timeit=True)

In [43]:
from math import pi, sin, cos, acos, tan, atan, ceil, radians, degrees, floor
import numpy as np
clearance = 0.05

In [44]:
def copier(solid, vect, number, distance, winkel):
    copies = []
    
    for i in range(number):
        copy = solid.rotate((0.0, 0.0, 0.0), (0.0, 0.0, 1.0), i * winkel)
        d = distance * i
        copy = copy.translate((vect[0] * d, vect[1] * d, vect[2] * d))
        copies.append(copy)
    
    return copies

def polar_to_cartesian(polvect):
    return (polvect[0] * cos(polvect[1]),
            polvect[0] * sin(polvect[1]))


#     Circle Involutes-Function:
#     Returns the Polar Coordinates of an Involute Circle
#     r = Radius of the Base Circle
#     rho = Rolling-angle in Degrees
def ev(r, rho):
    return (r / cos(rho), tan(rho) - rho)

def sphere_to_cartesian(vect):
    return (vect[0] * sin(vect[1]) * cos(vect[2]),  
            vect[0] * sin(vect[1]) * sin(vect[2]),
            vect[0] * cos(vect[1]))


In [ ]:

def rack(modul, length, height, width, pressure_angle=20.0, helix_angle=0.0):
    # Dimension Calculations
    pressure_angle = radians(pressure_angle)
    helix_angle = radians(helix_angle)
    
    modul = modul * (1.0 - clearance)
    c = modul / 6.0                                              # Tip Clearance
    mx = modul / cos(helix_angle)                                # Module Shift by Helix Angle in the X-Direction
    a = 2.0 * mx * tan(pressure_angle) + c * tan(pressure_angle) # Flank Width
    b = pi * mx / 2.0 - 2.0 * mx * tan(pressure_angle)           # Tip Width
    x = width * tan(helix_angle)                                 # Topside Shift by Helix Angle in the X-Direction
    nz = ceil((length + abs(2.0 * x)) / (pi * mx))               # Number of Teeth
    
    
    underside_points = [[0, -c],
                        [a, 2.0 * modul],
                        [a + b, 2.0 * modul],
                        [2.0 * a + b, -c],
                        [pi * mx, -c],
                        [pi * mx, modul - height],
                        [0.0, modul - height]]

    topside_points = [[0.0 + x, -c],
                      [a + x, 2.0 * modul],
                      [a + b + x, 2.0 * modul],
                      [2.0 * a + b + x, -c],
                      [pi * mx + x, -c],
                      [pi * mx + x, modul - height],
                      [0.0 + x, modul - height]]
    
    res = cq.Workplane('XY')
    res = res.polyline(underside_points).close()
    res = res.workplane(offset=width)
    res = res.polyline(topside_points).close()
    res = res.loft()
    
    teeth = copier(res, (1.0, 0.0, 0.0), nz, pi * mx, 0.0)
    res = teeth[0]
    for t in teeth[1:]:
        res = res.union(t)
        
    box = cq.Workplane().box(length, height + modul + 1.0, width + 1.0, centered=False)
    box = box.translate((abs(x), -height + modul - 0.5, -0.5))
    res = res.intersect(box)
    res = res.translate([-pi * mx * (nz - 1.0) / 2.0 - a - b / 2.0, -modul, 0.0])

    return res

show_object(rack(modul=1.0, length=60.0, height=5.0, width=20.0, pressure_angle=20.0, helix_angle=0.0))

In [ ]:

def mountable_rack(modul, length, height, width, pressure_angle, helix_angle, fasteners, profile, head):
    a_rack = rack(modul, length, height, width, pressure_angle, helix_angle)
    offset = length / fasteners;

    def ph_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile).extrude(profile * 0.6 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def cs_screw(idx):
        screw = cq.Workplane('XZ')
        screw = screw.circle(profile * 1.5 / 2.0).extrude(profile * 1.25 + modul * 2.25)
        screw = screw.faces('>Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def c_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def rc_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25 + profile / 4.0)
        screw = screw.circle(profile).workplane(offset=profile / 2.0)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25 - profile / 4.0)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    def css_screw(idx):
        screw = cq.Workplane('XZ').workplane(modul * 2.25)
        screw = screw.circle(profile).workplane(offset=profile * 0.6)
        screw = screw.circle(profile / 2.0).loft()
        screw = screw.faces('>Y').circle(profile).extrude(-modul * 2.25)
        screw = screw.faces('<Y').circle(profile / 2.0).extrude(height + modul)
        screw = screw.translate((idx * offset + (offset - length) / 2.0, modul, width / 2.0))
        return screw
    
    screw_f = {
        'PH': ph_screw,
        'CS': cs_screw,
        'C': c_screw,
        'RC': rc_screw,
        'CSS': css_screw,
    }[head]
    
    for i in range(fasteners):
        a_rack = a_rack.cut(screw_f(i))
    
    return a_rack

show_object(mountable_rack(modul=1.0, length=60.0, height=10.0, width=20.0,
                           pressure_angle=20.0, helix_angle=20.0, profile=3.0, head='CSS',fasteners=3))

In [102]:

def involute_gear_profile(module, teeth_number, pressure_angle=20.0, clearance=0.0):
    curve_points = 20
    m = module
    z = teeth_number
    a0 = np.radians(pressure_angle)
    c = clearance

    d0 = m * z         # pitch diameter
    da = m * (z + 2.0) # addendum circle diameter
    dd = m * (z - 2.0) - 2.0 * c # dedendum circle diameter
    s0 = m * (np.pi / 2.0) # tooth thickness on the pitch circle
    inv_a0 = tan(a0) - a0 # pressure angle involute

    r0 = d0 / 2.0 # pitch circle radius
    ra = da / 2.0 # addendum circle radius
    rd = dd / 2.0 # dedendum circle radius
    rb = np.cos(a0) * r0 # base circle radius
    
    tau = np.pi * 2.0 / z
    
    def calc_phi(r):
        cos_a = r0 / r * np.cos(a0)
        a = np.arccos(cos_a)
        inv_a = np.tan(a) - a
        s = r * (s0 / d0 + inv_a0 - inv_a)
        return s / r
    
    def rotate_point(x, y, angle):
        rx = x * np.cos(angle) - y * np.sin(angle)
        ry = x * np.sin(angle) + y * np.cos(angle)
        return rx, ry
    
    def involute_curve(r, angle, mirror):        
        phi = calc_phi(r)
        if mirror:
            phi = -phi
        x = np.cos(phi) * r
        y = np.sin(phi) * r
        
        return rotate_point(x, y, angle)
    
    profile = cq.Workplane('XY')
#     profile = profile.circle(rd, False)

    rr = max(rb, rd)
    
    angle = 0.0
    for i in range(z):
        
        profile = profile.parametricCurve(lambda r: involute_curve(r, angle, False),
                                          curve_points, rr, ra)
        arc_sp = involute_curve(ra, angle, False)
        arc_ep = involute_curve(ra, angle, True)
        
        profile = profile.moveTo(*arc_sp).radiusArc(arc_ep, ra)
        profile = profile.parametricCurve(lambda r: involute_curve(r, angle, True),
                                          curve_points, rr, ra)
        
        arc_sp = involute_curve(rr, angle, True)
        rho = angle - tau * 0.5
        arc_mp = (np.cos(rho) * rd, np.sin(rho) * rd)
        arc_ep = involute_curve(rr, angle - tau, False)
        
        profile = profile.moveTo(*arc_sp).threePointArc(arc_mp, arc_ep)
        
        angle -= tau

    profile = profile.consolidateWires()
        
    return profile

profile = involute_gear_profile(1.0, 8)
# profile = profile.twistExtrude(10.0, 150.0)
# profile = profile.extrude(10.0)

show(profile)


| | Object: Group
| | | discretize time:    0.00 sec
| | | edge list:        0.01 sec
| | shape render time:    0.01 sec
| overall render time:    0.01 sec
| create bounding box:    0.00 sec
| configure view:       0.04 sec
add shapes:             0.06 sec
configure display:      0.02 sec
Done, using side car 'Gears'


In [116]:
def spur_gear(module, teeth_number, width, bore, pressure_angle=20.0, helix_angle=0.0, optimized=True):
    # Dimension Calculations
    helix_angle = radians(helix_angle)    
    d = module * teeth_number                               # Pitch Circle Diameter
    r = d / 2                                               # Pitch Circle Radius
    c = 0.0 if teeth_number < 3 else module / 6.0           # Tip Clearance
    df = d - 2.0 * (module + c)                             # Root Circle Diameter
    rf = df / 2.0                                           # Root Radius
    gamma = width / (r * np.tan(np.pi / 2.0 - helix_angle)) # Torsion Angle for Extrusion

    r_hole = (2.0 * rf - bore) / 8.0                        # Radius of Holes for Material-/Weight-Saving
    rm = bore / 2.0 + 2.0 * r_hole                          # Distance of the Axes of the Holes from the Main Axis
    z_hole = floor(2 * pi * rm / (3.0 * r_hole))            # Number of Holes for Material-/Weight-Saving
    
    res = involute_gear_profile(module, teeth_number, pressure_angle)
    res = res.twistExtrude(width, np.degrees(gamma)) 

    # "bore"
    res = res.faces('<Z').circle(rm + r_hole * 1.49).cutThruAll()
    
    body = cq.Workplane('XY')
    if optimized:
        res = (res.faces('<Z').
               circle((bore + r_hole) / 2.0).
               circle(bore / 2.0).
               extrude(width))
        
        if (width - r_hole / 2.0) < (width * 2.0 / 3.0):
            ht = width * 2.0 / 3.0
        else:
            ht = width - r_hole / 2.0
    
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle((bore + r_hole) / 2.0)
        for i in range(z_hole):
            pos = sphere_to_cartesian((rm, pi / 2.0, i * pi * 2.0 / z_hole))
            body = body.moveTo(pos[0], pos[1]).circle(r_hole)
        body = body.consolidateWires().extrude(ht)
    else:
        body = body.circle(rm + r_hole * 1.51)
        body = body.circle(bore / 2.0)
        body = body.consolidateWires().extrude(width)

    res = res.union(body)
    return res
    
gear = spur_gear(module=1.0, teeth_number=60, width=5.0, bore=5.0, pressure_angle=20.0, helix_angle=20.0, optimized=True)
show(gear)


| | Object: Group
| | | (Caching 140391823795120)
| | | build mesh time:    0.84 sec
| | | discretize time:    0.03 sec
| | | edge list:        0.01 sec
| | shape render time:    0.89 sec
| overall render time:    0.90 sec
| create bounding box:    0.23 sec
| configure view:       0.04 sec
add shapes:             1.17 sec
configure display:      0.01 sec
Done, using side car 'Gears'
